In [1]:
# Basic imports
import os
import sys
import time
import timeit
import random
import pickle
import re
from itertools import product
import warnings

# System path modification
sys.path.insert(0, '..')

# Data handling
import pandas as pd
import numpy as np

# Machine learning imports
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer, MissingIndicator
from sklearn.model_selection import (
    train_test_split, KFold, StratifiedKFold, GroupKFold, StratifiedGroupKFold, LeaveOneOut, cross_validate, cross_val_score
)
from sklearn.linear_model import (
    LinearRegression, Lasso, LassoCV, MultiTaskLasso, MultiTaskLassoCV,
    ElasticNet, ElasticNetCV, MultiTaskElasticNet, MultiTaskElasticNetCV
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

from sklearn.cross_decomposition import PLSRegression
from sklearn.inspection import permutation_importance

# Statistic imports 
from scipy.stats import ks_2samp
from scipy.special import kl_div
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.spatial.distance import pdist, squareform

# Specialized imputation and visualization packages
import miceforest as mf
import missingno as msno
#from missforest import MissForest
#import magic
from src.gain import *

# Custom modules
from src.train import *
from src.functions import *
from src.plots import *
from src.dataset import *
from src.multixgboost import *
from src.wrapper import *
from src.debug import *

# Visualizatiokn 
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning and machine learning specific 
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
import xgboost as xgb
import shap

from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.models import (
    GatedAdditiveTreeEnsembleConfig,
    DANetConfig,
    TabTransformerConfig,
    TabNetModelConfig,
)

# Ignore warnings
warnings.filterwarnings("ignore")

# Print CUDA availability for PyTorch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

from omegaconf import DictConfig
torch.serialization.safe_globals([DictConfig])

False
0


## Load data 

In [2]:
data = load_pickle_data_palettes()

results_pickle_folder = "../pickle/"

# Unpack data
df_X, df_y, df_all, df_FinalCombination = data["df_X"], data["df_y"], data["df_all"], data["df_FinalCombination"]
dict_select = data["dict_select"]

# Unpack colormaps
full_palette, gender_palette, dx_palette = data["colormaps"].values()

In [3]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True

print(sum(idx_test))

print(df_all[idx_test].RID)

df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("category")

test_indices = [i for i, val in enumerate(idx_test) if val]

13
3609    2002
5631    4167
5662    4176
5780    4215
5950    4349
6069    4292
6077    4453
6085    4489
6224    4505
6400    4576
6429    4300
7021    2374
7192    4179
Name: RID, dtype: int64


# Leave-One-Complete-Out (LOCO-CV)

## All features

In [4]:
random_state=42

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("KNNImputer", KNNImputer(n_neighbors=1)),
]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("SimpleImputer_most_frequent", SimpleImputer(strategy="most_frequent")),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.01, 'l1_ratio': 0.01})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.5079831261101071, 'learning_rate': 0.0769592094304232, 'max_depth': 6, 'min_child_weight': 7, 'subsample': 0.8049983288913105})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [5]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y.columns.tolist(),
    continuous_cols=continuous_features,
    categorical_cols=ordinal_features
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=False,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__

predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]

In [6]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: LinearRegression
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskElasticNet
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskElasticNet_tuned
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskLasso
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskLasso_tuned
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: RandomForestRegressor
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: XGBoostRegressor
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: XGBoostRegressor_tuned
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: TabNetRegressor_default
Continuous Imputer: KNNImputer, Ordinal Imputer: S

In [7]:
# Initialize HDF5 file
results_file = '../pickle/training_3_loonona_dict_results.pickle'

In [8]:
if os.path.exists(results_file): 
    with open(results_file, "rb") as input_file:
        all_dict_results = pickle.load(input_file)

else : 
    all_dict_results = []

In [9]:
if False : 
    params_comb = [{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}]

    for params in params_comb:
        all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"params": params})

In [10]:
all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"fitting_time":None})

In [11]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    params = {
            "ordinal_imputer": name_ordinal_imputer, 
            "continuous_imputer": name_continuous_imputer, 
            "model": name_model, "train_shape" : [df_X.shape[0]-1, df_X.shape[1]],
            "test_shape": [1, df_X.shape[1]]
        }
    
    # Define the subset of keys you care about
    keys_to_check = ['ordinal_imputer', 'continuous_imputer', 'model']  # or whatever subset you want

    # Check if a result in all_dict_results has the same values for just those keys
    if any(all(result['params'].get(k) == params.get(k) for k in keys_to_check) for result in all_dict_results):
        print(f"Skipping existing combination (subset match): {[params[k] for k in keys_to_check]}")
        continue

    dict_results = {
            "params": params, 
            "imputation_time": [],
            "fitting_time": [], 
            "results_adj": [], 
            "results_org": []
        }

    for test_nloc in test_indices: 
        print(test_nloc)

        idx_train = [True for i in range(df_X.shape[0])]
        idx_test = [False for i in range(df_X.shape[0])]

        idx_test[test_nloc] = True
        idx_train[test_nloc] = False

        df_X_train = df_X.loc[idx_train]
        df_X_test = df_X.loc[idx_test]

        df_y_train = df_y.loc[idx_train]
        df_y_test = df_y.loc[idx_test]

        c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
        c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

        try: 
        
            # Now you can call your `train_model` function with these components
            fold_dict_results = train_imputer_model(
                df_X_train, df_X_test, df_y_train, df_y_test,
                c_train, c_test,
                ordinal_imputer_instance, name_ordinal_imputer,
                continuous_imputer_instance, name_continuous_imputer,
                model_instance, name_model,
                separate_imputers=True  # Or however you want to specify
            )
            
            dict_results["imputation_time"].append(fold_dict_results["imputation_time"]) 
            dict_results["fitting_time"].append(fold_dict_results["fitting_time"])  
            dict_results["results_adj"].append(fold_dict_results["results_adj"])  
            dict_results["results_org"].append(fold_dict_results["results_org"])  

        except Exception as e:  

            print(e)
            
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)

    # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)


Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'LinearRegression']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskElasticNet']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskElasticNet_tuned']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskLasso']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskLasso_tuned']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'RandomForestRegressor']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'XGBoostRegressor']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'XGBoostRegressor_tuned']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', '

In [12]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
results_file

'../pickle/training_3_loonona_dict_results.pickle'

In [14]:
with open(results_file, "rb") as input_file:
    dict_results_loo_nona = pickle.load(input_file)

In [15]:
df_results_nona = pd.json_normalize(dict_results_loo_nona)

# Train only on MRI features

In [17]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True

In [18]:
df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("category")

In [19]:
df_X_train = df_X[dict_select["MRIth"]].loc[idx_train]
df_X_test = df_X[dict_select["MRIth"]].loc[idx_test]

df_y_train = df_y.loc[idx_train]
df_y_test = df_y.loc[idx_test]

c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

In [20]:
random_state=42
n_imputation_iter = 10

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("NoImputer", KNNImputer(n_neighbors=1)),

]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("NoImputer", SimpleImputer(strategy="most_frequent")),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.01, 'l1_ratio': 0.01})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.5079831261101071, 'learning_rate': 0.0769592094304232, 'max_depth': 6, 'min_child_weight': 7, 'subsample': 0.8049983288913105})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [21]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X_train.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y_train.columns.tolist(),
    continuous_cols=dict_select["MRIth"],
    categorical_cols=[]
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=True,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__
predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]


In [22]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: LinearRegression
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: RandomForestRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_default
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_custom
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: PLSRegression_4_components
Continuous Imputer: NoImputer, Ordinal 

In [23]:
# Initialize HDF5 file
results_file = '../pickle/training_3_loonona_dict_results.pickle'

with open('../pickle/training_3_loonona_dict_results.pickle', "rb") as input_file:
    all_dict_results = pickle.load(input_file)

In [ ]:
all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"fitting_time":None})

In [27]:
for res in all_dict_results: 
    print(res["params"])

{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'LinearRegression', 'train_shape': [2893, 348], 'test_shape': [1, 348]}
{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'MultiTaskElasticNet', 'train_shape': [2893, 348], 'test_shape': [1, 348]}
{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'MultiTaskElasticNet_tuned', 'train_shape': [2893, 348], 'test_shape': [1, 348]}
{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'MultiTaskLasso', 'train_shape': [2893, 348], 'test_shape': [1, 348]}
{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'MultiTaskLasso_tuned', 'train_shape': [2893, 348], 'test_shape': [1, 348]}
{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'RandomForestRegressor', 'train_shape': [2893, 348

In [28]:
from src.debug import *

rm_combinations = [{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'LinearRegression', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'MultiTaskElasticNet', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'MultiTaskLasso', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'RandomForestRegressor', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'XGBoostRegressor', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'XGBoostRegressor_tuned', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetRegressor_default', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetRegressor_custom', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'PLSRegression_4_components', 'train_shape': (2881, 200), 'test_shape': (13, 200)},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'MultiTaskElasticNet_tuned', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'MultiTaskLasso_tuned', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
{'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}, 
]
if False : 
    for par in rm_combinations:
        all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"params":par})

In [30]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    params = {
            "ordinal_imputer": name_ordinal_imputer, 
            "continuous_imputer": name_continuous_imputer, 
            "model": name_model, "train_shape" : [df_X.shape[0]-1, df_X.shape[1]],
            "test_shape": [1, df_X.shape[1]]
        }
    
    # Define the subset of keys you care about
    keys_to_check = ['ordinal_imputer', 'continuous_imputer', 'model']  # or whatever subset you want

    # Check if a result in all_dict_results has the same values for just those keys
    if any(all(result['params'].get(k) == params.get(k) for k in keys_to_check) for result in all_dict_results):
        print(f"Skipping existing combination (subset match): {[params[k] for k in keys_to_check]}")
        continue

    dict_results = {
            "params": params, 
            "imputation_time": [],
            "fitting_time": [], 
            "results_adj": [], 
            "results_org": []
        }

    for test_nloc in test_indices: 
        print(test_nloc)

        idx_train = [True for i in range(df_X.shape[0])]
        idx_test = [False for i in range(df_X.shape[0])]

        idx_test[test_nloc] = True
        idx_train[test_nloc] = False

        df_X_train = df_X[dict_select["MRIth"]].loc[idx_train]
        df_X_test = df_X[dict_select["MRIth"]].loc[idx_test]

        df_y_train = df_y.loc[idx_train]
        df_y_test = df_y.loc[idx_test]

        c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
        c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

        try: 
        
            # Now you can call your `train_model` function with these components
            fold_dict_results = train_imputer_model(
                df_X_train, df_X_test, df_y_train, df_y_test,
                c_train, c_test,
                ordinal_imputer_instance, name_ordinal_imputer,
                continuous_imputer_instance, name_continuous_imputer,
                model_instance, name_model,
                separate_imputers=True  # Or however you want to specify
            )
            
            dict_results["imputation_time"].append(fold_dict_results["imputation_time"]) 
            dict_results["fitting_time"].append(fold_dict_results["fitting_time"])  
            dict_results["results_adj"].append(fold_dict_results["results_adj"])  
            dict_results["results_org"].append(fold_dict_results["results_org"])  

        except Exception as e:  

            print(e)
            
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)

    # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)


1790
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Saving predictions in dict!
2390
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Saving predictions in dict!
2401
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Saving predictions in dict!
2437
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Saving predictions in dict!
2493
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Saving predictions in dict!
2533
Using separate imputers for ordinal and continuous data.
No NaN in train data -> Keep as it is. 
No NaN in test data -> Keep as it is. 
Saving predict

In [31]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
with open(results_file, "rb") as input_file:
    all_dict_results = pickle.load(input_file)

## Print Table for paper

In [4]:
with open('../pickle/training_3_loonona_dict_results.pickle', "rb") as input_file:
    dict_results_loo_nona = pickle.load(input_file)

In [110]:
def generate_metric_table(
results_list,
    targets,
    metric_name="r2",
    source="Adjusted",
    csv_filename=None,
    sort_order="descending"
):
    key = "results_adj" if source.lower() == "adjusted" else "results_org"
    df_rows = []

    for res in results_list:
        result_blocks = res.get(key, [])
        if not isinstance(result_blocks, list) or len(result_blocks) == 0:
            continue

        y_preds_all = {t: [] for t in targets}
        y_tests_all = {t: [] for t in targets}

        for fold in result_blocks:
            y_pred = fold["y_pred"]
            y_test = fold["y_test"]
            if y_pred.shape != y_test.shape:
                continue

            for i, target in enumerate(targets):
                if y_pred.shape[1] <= i:
                    continue
                y_preds_all[target].append(y_pred[0, i])
                y_tests_all[target].append(y_test[0, i])

        target_metrics = []
        for target in targets:
            y_preds = y_preds_all[target]
            y_tests = y_tests_all[target]

            if len(y_preds) < 2:
                metric = float("nan")
            else:
                if metric_name == "r2":
                    metric = r2_score(y_tests, y_preds)
                elif metric_name == "mae":
                    metric = mean_absolute_error(y_tests, y_preds)
                elif metric_name == "mse":
                    metric = mean_squared_error(y_tests, y_preds)
                elif metric_name == "corr":
                    metric = pearsonr(y_tests, y_preds)[0]
                else:
                    raise ValueError(f"Unsupported metric: {metric_name}")
            target_metrics.append(metric)

        # Mean ± std across targets
        mean_metric = np.nanmean(target_metrics)
        std_metric = np.nanstd(target_metrics)

        # Time stats
        imp_times = np.array(res.get("imputation_time", []), dtype=np.float64)
        fit_times = np.array(res.get("fitting_time", []), dtype=np.float64)

        imp_time_mean = np.nanmean(imp_times)
        imp_time_std = np.nanstd(imp_times)

        fit_time_mean = np.nanmean(fit_times)
        fit_time_std = np.nanstd(fit_times)

        params = res.get("params", {})
        df_rows.append({
            "Ordinal Imputer": params.get("ordinal_imputer"),
            "Continuous Imputer": params.get("continuous_imputer"),
            "Model": params.get("model"),
            **{target: m for target, m in zip(targets, target_metrics)},
            "Mean ± Std": f"{mean_metric:.3f} ± {std_metric:.3f}",
            "Imputation Time": f"{imp_time_mean:.2f} ± {imp_time_std:.2f}",
            "Fitting Time": f"{fit_time_mean:.2f} ± {fit_time_std:.2f}"
        })

    df = pd.DataFrame(df_rows)

    # Optional: sort
    if sort_order and targets:
        df = df.sort_values(by=targets[0], ascending=(sort_order == "ascending"))

    # Optional: Save to CSV
    if csv_filename:
        df.to_csv(csv_filename, index=False)

    # Format for LaTeX
    latex_df = df.copy()
    for col in targets:
        latex_df[col] = latex_df[col].apply(lambda x: f"{x:.3f}" if pd.notnull(x) else "–")
    latex_output = latex_df.to_latex(index=False, escape=False)

    return df, latex_output

In [111]:
latex_df, latex_corr = generate_metric_table(
    results_list=dict_results_loo_nona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='corr',
    source="Adjusted",
    csv_filename="../tables/3_training_loonona_corr_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_corr)

\begin{tabular}{llllllllll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± Std & Imputation Time & Fitting Time \\
\midrule
SimpleImputer_most_frequent & KNNImputer & GatedAdditiveTreeEnsembleConfig_tab & 0.701 & 0.774 & 0.245 & 0.823 & 0.636 ± 0.230 & 3.50 ± 0.07 & 17.88 ± 0.19 \\
SimpleImputer_most_frequent & KNNImputer & RandomForestRegressor & 0.686 & 0.653 & 0.297 & 0.763 & 0.600 ± 0.180 & 3.38 ± 0.07 & 52.01 ± 0.67 \\
SimpleImputer_most_frequent & KNNImputer & TabNetRegressor_default & 0.637 & 0.642 & 0.682 & 0.529 & 0.622 ± 0.057 & 3.40 ± 0.19 & 13.19 ± 0.90 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskLasso_tuned & 0.606 & 0.717 & 0.343 & 0.676 & 0.585 ± 0.146 & 3.53 ± 0.05 & 1.51 ± 0.01 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskElasticNet_tuned & 0.597 & 0.714 & 0.346 & 0.672 & 0.582 ± 0.143 & 3.50 ± 0.03 & 1.50 ± 0.01 \\
SimpleImputer_most_frequent & KNNImputer & LinearRegression & 0.591 & 0.705 

In [112]:
latex_df, latex_r2 = generate_metric_table(
    results_list=dict_results_loo_nona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='r2',
    source="Adjusted",
    csv_filename="../tables/3_training_loonona_r2_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_r2)

\begin{tabular}{llllllllll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± Std & Imputation Time & Fitting Time \\
\midrule
SimpleImputer_most_frequent & KNNImputer & MultiTaskLasso_tuned & 0.322 & 0.475 & 0.088 & 0.304 & 0.297 ± 0.138 & 3.53 ± 0.05 & 1.51 ± 0.01 \\
SimpleImputer_most_frequent & KNNImputer & LinearRegression & 0.314 & 0.463 & 0.109 & 0.316 & 0.301 ± 0.126 & 3.42 ± 0.11 & 0.15 ± 0.06 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskElasticNet_tuned & 0.312 & 0.471 & 0.099 & 0.294 & 0.294 ± 0.132 & 3.50 ± 0.03 & 1.50 ± 0.01 \\
SimpleImputer_most_frequent & KNNImputer & RandomForestRegressor & 0.200 & 0.328 & 0.057 & 0.214 & 0.200 ± 0.096 & 3.38 ± 0.07 & 52.01 ± 0.67 \\
NoImputer & NoImputer & RandomForestRegressor & 0.100 & 0.169 & -0.168 & 0.113 & 0.054 ± 0.131 & nan ± nan & 32.37 ± 0.24 \\
SimpleImputer_most_frequent & KNNImputer & TabTransformerConfig_tab & 0.057 & -0.476 & -1.514 & -0.647 & -0.645 ± 0.56

In [115]:
latex_df, latex_mae = generate_metric_table(
    results_list=dict_results_loo_nona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mae',
    source="Adjusted",
    csv_filename="../tables/3_training_loonona_mae_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mae)

\begin{tabular}{llllllllll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± Std & Imputation Time & Fitting Time \\
\midrule
SimpleImputer_most_frequent & KNNImputer & MultiTaskLasso_tuned & 0.633 & 0.547 & 0.584 & 0.596 & 0.590 ± 0.031 & 3.53 ± 0.05 & 1.51 ± 0.01 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskElasticNet_tuned & 0.639 & 0.551 & 0.587 & 0.600 & 0.594 ± 0.032 & 3.50 ± 0.03 & 1.50 ± 0.01 \\
SimpleImputer_most_frequent & KNNImputer & LinearRegression & 0.649 & 0.546 & 0.571 & 0.603 & 0.592 ± 0.039 & 3.42 ± 0.11 & 0.15 ± 0.06 \\
SimpleImputer_most_frequent & KNNImputer & RandomForestRegressor & 0.666 & 0.669 & 0.645 & 0.603 & 0.646 ± 0.026 & 3.38 ± 0.07 & 52.01 ± 0.67 \\
NoImputer & NoImputer & RandomForestRegressor & 0.709 & 0.727 & 0.699 & 0.585 & 0.680 ± 0.056 & nan ± nan & 32.37 ± 0.24 \\
NoImputer & NoImputer & MultiTaskLasso_tuned & 0.725 & 0.699 & 0.711 & 0.695 & 0.708 ± 0.012 & nan ± nan & 0.83 ± 0.01 

In [116]:
latex_df, latex_mse = generate_metric_table(
    results_list=dict_results_loo_nona,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mse',
    source="Adjusted",
    csv_filename="../tables/3_training_loonona_mse_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mse)

\begin{tabular}{llllllllll}
\toprule
Ordinal Imputer & Continuous Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± Std & Imputation Time & Fitting Time \\
\midrule
SimpleImputer_most_frequent & KNNImputer & MultiTaskLasso_tuned & 0.672 & 0.463 & 0.438 & 0.572 & 0.536 ± 0.093 & 3.53 ± 0.05 & 1.51 ± 0.01 \\
SimpleImputer_most_frequent & KNNImputer & LinearRegression & 0.679 & 0.473 & 0.428 & 0.562 & 0.536 ± 0.096 & 3.42 ± 0.11 & 0.15 ± 0.06 \\
SimpleImputer_most_frequent & KNNImputer & MultiTaskElasticNet_tuned & 0.682 & 0.467 & 0.432 & 0.581 & 0.541 ± 0.098 & 3.50 ± 0.03 & 1.50 ± 0.01 \\
SimpleImputer_most_frequent & KNNImputer & RandomForestRegressor & 0.793 & 0.593 & 0.452 & 0.646 & 0.621 ± 0.122 & 3.38 ± 0.07 & 52.01 ± 0.67 \\
NoImputer & NoImputer & RandomForestRegressor & 0.892 & 0.733 & 0.561 & 0.729 & 0.729 ± 0.117 & nan ± nan & 32.37 ± 0.24 \\
SimpleImputer_most_frequent & KNNImputer & TabTransformerConfig_tab & 0.935 & 1.303 & 1.207 & 1.355 & 1.200 ± 0.162 & 3